# test note


* jupyterはコンテナ起動すること
* テストベッド一式起動済みであること


In [1]:
import requests
from pathlib import Path
import zipfile
from glob import glob
import pprint
import json
import time

# entry point address
is_container = False
if is_container:
    backend_entry_point = 'http://host.docker.internal:8888/qai-testbed/api/0.0.1'
    ip_entry_point = 'http://host.docker.internal:8888/qai-ip/api/0.0.1'
else:
    backend_entry_point = 'http://host.docker.internal:5000/qai-testbed/api/0.0.1'
    ip_entry_point = 'http://host.docker.internal:6000/qai-ip/api/0.0.1'

current_dir = %pwd
ait_name='eval_handwrite_acc_tf2.3_0.1'
ait_dir = Path(current_dir).joinpath(ait_name)

def get_bk(path:str, is_print_response:bool = True, is_print_json:bool = True):
    response = requests.get(f'{backend_entry_point}{path}')
    if is_print_response:
        print(response)
    if is_print_json:
        pprint.pprint(response.json())
    return response

def get_ip(path:str, is_print_response:bool = True, is_print_json:bool = True):
    response = requests.get(f'{ip_entry_point}{path}')
    if is_print_response:
        print(response)
    if is_print_json:
        pprint.pprint(response.json())
    return response

def post_bk(path:str, data):
    response = requests.post(f'{backend_entry_point}{path}', 
                             json.dumps(data),
                             headers={'Content-Type': 'application/json'}) 
    print(response)  
    pprint.pprint(response.json())
    return response

def post_ip(path:str, data):
    response = requests.post(f'{ip_entry_point}{path}', 
                             json.dumps(data),
                             headers={'Content-Type': 'application/json'}) 
    print(response)  
    pprint.pprint(response.json())
    return response
    
def uploadzip_id(zip_path:str):
    fileobj = open(zip_path, 'rb')
    response = requests.post(f'{ip_entry_point}/deploy-dag', 
                             data = {"name":"dag_zip"}, 
                             files={"archive": (str(Path(zip_path).name), fileobj)})
    print(response)
    pprint.pprint(response.json())
    return response

def create_zip():
    zip_dir = ait_dir
    zip_file = f'{ait_dir}.zip'

    with zipfile.ZipFile(zip_file, 'w', compression=zipfile.ZIP_STORED) as new_zip:
        new_zip.write(str(zip_dir.joinpath('dag.py')), arcname=f'{ait_name}/dag.py')
        new_zip.write(str(zip_dir.joinpath('dev').joinpath('dockerfile')), arcname=f'{ait_name}/dev/dockerfile')
        
        dir_path = zip_dir.joinpath('dev').joinpath('repository')
        files = glob(str(dir_path.joinpath('**')), recursive=True)
        for file in files:
            file_name = Path(file).name
            new_zip.write(file, arcname=f'{ait_name}/dev/repository/{file_name}')
    return zip_file

def build_ait(zip_path):
    zip_name = str(Path(zip_path).name)
    with open(zip_path, 'rb') as file_obj:
        response = requests.post(f'{ip_entry_point}/deploy-dag', 
                                 files={"dag_zip": (zip_name, file_obj)})
    print(response)
    pprint.pprint(response.json())
    return response
    
def post_manifest(manifest_path):
    manifest_name = str(Path(manifest_path).name)
    with open(manifest_path, 'rb') as file_obj:
        response = requests.post(f'{backend_entry_point}/testRunners/manifest', 
                                 files={"ait.manifest": (manifest_name, file_obj)})
    print(response)
    pprint.pprint(response.json())
    return response

def _find_file(dir_path: Path, file_name: str):
    files = glob(str(dir_path.joinpath(f'**/{file_name}')), recursive=True)
    if len(files) == 0:
        raise Exception(f'not found {file_name} in zip.')
    elif len(files) > 1:
        raise Exception(f'{file_name} must be one exists in zip.')
    return files[0]

def deploy_ait(need_build_container:bool=True):    
    # deploy
    if need_build_container:
        zip_file = create_zip()
        build_ait(zip_file)

    # add manifest
    post_manifest(_find_file(ait_dir, 'ait.manifest.json'))

def wait_run_complete(wait_time:float=1):
    # animation = "|/-\\"
    animation = [
        " [*     ]",
        " [ *    ]",
        " [  *   ]",
        " [   *  ]",
        " [    * ]",
        " [     *]",
        " [    * ]",
        " [   *  ]",
        " [  *   ]",
        " [ *    ]",
    ]
    i = 0

    while True:
        print(animation[i % len(animation)], end="\r")
        i += 1

        run_status = get_bk('/dep-a/mlComponents/1/testDescriotions/run-status', is_print_response=False, is_print_json=False).json()
        if run_status['Job']['Status'] != 'RUNNING':
            pprint.pprint(run_status['Runs'])
            break
        time.sleep(wait_time)


In [2]:
# health check

get_bk('/health-check')
get_ip('/health-check')

<Response [200]>
{'Code': 0, 'Message': 'alive.'}
<Response [200]>
{'Code': 0, 'Message': 'alive.'}


<Response [200]>

In [3]:
# deploy AIT

deploy_ait(need_build_container=False)

<Response [200]>
{'Code': 'M00001', 'Message': 'Add AIT manifest success'}


In [4]:
# add inventories

post_bk('/dep-a/mlComponents/1/inventories', {
    'Name': 'trained_model_1',
    'Type': "model", 'FileSystem': "UNIX_FILE_SYSTEM",
    'Address': r'C:\m-akita\github\qai-testbed\ait_repository\eval_handwrite_acc_tf2.3_0.1\exp\model_1.h5',
    'Description': "Tensorflow 2.3で学習したモデル_1",
    'Formats': ["h5"],
})
post_bk('/dep-a/mlComponents/1/inventories', {
    'Name': 'test_set_images',
    'Type': "dataset", 'FileSystem': "UNIX_FILE_SYSTEM",
    'Address': r'C:\m-akita\github\qai-testbed\ait_repository\eval_handwrite_acc_tf2.3_0.1\local_qai\inventory\test_set_images\t10k-images-idx3-ubyte.gz',
    'Description': "テスト画像セット（MNISTフォーマット）",
    'Formats': ["gz"],
})
post_bk('/dep-a/mlComponents/1/inventories', {
    'Name': 'test_set_labels',
    'Type': "dataset", 'FileSystem': "UNIX_FILE_SYSTEM",
    'Address': r'C:\m-akita\github\qai-testbed\ait_repository\eval_handwrite_acc_tf2.3_0.1\local_qai\inventory\test_set_labels\t10k-labels-idx1-ubyte.gz',
    'Description': "テスト画像ラベル（MNISTフォーマット）",
    'Formats': ["gz"],
})

<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}
<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}
<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}


<Response [200]>

In [5]:
# get ait_json and inventory_jsons

res_json = get_bk('/testRunners', is_print_json=False).json()
ait_json = [j for j in res_json['TestRunners'] if j['Name'] == 'eval_handwrite_acc_tf2.3'][0]

res_json = get_bk('/dep-a/mlComponents/1/inventories', is_print_json=False).json()
inv_1_json = [j for j in res_json['Inventories'] if j['Name'] == 'trained_model_1'][0]
inv_4_json = [j for j in res_json['Inventories'] if j['Name'] == 'test_set_images'][0]
inv_5_json = [j for j in res_json['Inventories'] if j['Name'] == 'test_set_labels'][0]

<Response [200]>
<Response [200]>


In [6]:
# add teast_descriptions

post_bk('/dep-a/mlComponents/1/testDescriotions', {
    "Name": "eval_handwrite_acc_tf2.3_1",
    "QualityDimensionID": 5,
    "QualityMeasurements": [
        {"Id":4, "Value":"0.95", "RelationalOperatorId":5},
        {"Id":5, "Value":"0.95", "RelationalOperatorId":5},
        {"Id":6, "Value":"0.95", "RelationalOperatorId":5},
        {"Id":7, "Value":"0.95", "RelationalOperatorId":5},
        {"Id":8, "Value":"0.95", "RelationalOperatorId":5},
        {"Id":9, "Value":"0.95", "RelationalOperatorId":5},
        {"Id":10, "Value":"0.95", "RelationalOperatorId":5},
        {"Id":11, "Value":"0.95", "RelationalOperatorId":5},
        {"Id":12, "Value":"0.95", "RelationalOperatorId":5}
    ],
    "TargetInventories": [
        {"Id":1, "InventoryId": inv_1_json['Id'], "TemplateInventoryId": 7},
        {"Id":2, "InventoryId": inv_4_json['Id'], "TemplateInventoryId": 8},
        {"Id":2, "InventoryId": inv_5_json['Id'], "TemplateInventoryId": 9}
    ],
    "TestRunner": {
        "Id":ait_json['Id'],
        "Params":[
            {"TestRunnerParamTemplateId":12, "Value":"10"},
            {"TestRunnerParamTemplateId":13, "Value":"28"},
            {"TestRunnerParamTemplateId":14, "Value":"macro"},
            {"TestRunnerParamTemplateId":15, "Value":"raise"}
        ]
    }
})

<Response [200]>
{'Result': {'Code': 'T22000', 'Message': 'append test description success.'}}


<Response [200]>

In [7]:
# get test_description_jsons

res_json = get_bk('/dep-a/mlComponents/1/testDescriotions', is_print_json=False).json()
td_1_json = [j for j in res_json['Test']['TestDescriptions'] if j['Name'] == 'eval_handwrite_acc_tf2.3_1'][0]

<Response [200]>


In [11]:
# run test_descriptions

post_bk('/dep-a/mlComponents/1/testDescriotions/runners', {
    "Command": "AsyncStart",
    "TestDescriptionIds": [td_1_json['Id']]
})

wait_run_complete(wait_time=0.5)

<Response [200]>
{'Job': {'Id': '2', 'StartDateTime': '2020-08-25 23:40:16.171288+09:00'},
 'Result': {'Code': 'R12000', 'Message': 'job launch success.'}}
[{'Id': 2,
  'Result': 'OK',
  'ResultDetail': '{\n'
                  '  "Accuracy": true,\n'
                  '  "Precision": true,\n'
                  '  "Recall": true,\n'
                  '  "F−measure": true,\n'
                  '  "AccuracyByClass": true,\n'
                  '  "PrecisionByClass": true,\n'
                  '  "RecallByClass": true,\n'
                  '  "F−measureByClass": true,\n'
                  '  "AUC": true\n'
                  '}',
  'Status': 'DONE',
  'TestDescriptionID': 4}]


In [12]:
res_json = get_bk('/dep-a/mlComponents/1/testDescriotions/{}'.format(td_1_json['Id']), is_print_json=False).json()
pprint.pprint(res_json)

<Response [200]>
{'Result': {'Code': 'T32000', 'Message': 'get detail success.'},
 'TestDescriptionDetail': {'Id': 4,
                           'Name': 'eval_handwrite_acc_tf2.3_1',
                           'Opinion': '',
                           'QualityDimension': {'Id': 5, 'Name': '学習の正確性'},
                           'QualityMeasurements': [{'Description': '(TP+TN) / '
                                                                   '(TP+FP+FN+TN)',
                                                    'Id': 4,
                                                    'Name': 'Accuracy',
                                                    'RelationalOperatorId': 5,
                                                    'Structure': 'single',
                                                    'Value': '0.95'},
                                                   {'Description': 'TP / '
                                                                   '(TP+FP)',
                         

In [13]:
# generate report

res = post_bk('/dep-a/mlComponents/1/testDescriotions/reportGenerator', {
    "Command": "Generate",
    "Destination": [str(td_1_json['Id'])]
})
pprint.pprint(res.json)

<Response [200]>
{'OutParams': {'ReportUrl': 'http://127.0.0.1:5000/qai-testbed/api/0.0.1/download/17'},
 'Result': {'Code': 'D12000', 'Message': 'command invoke success.'}}
<bound method Response.json of <Response [200]>>
